In [53]:
import pandas as pd
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt
import datetime, time
import os
import json
from dotenv import load_dotenv

In [54]:

def establish_db_connection(server, database, username, password, driver):
    connection_string = (
        f"mssql+pyodbc://{username}:{password}@{server}/{database}"
        f"?driver={driver.replace(' ', '+')}"
    )
    engine = create_engine(connection_string)

    try:
        with engine.connect() as conn:
            result = conn.execute(text("SELECT @@VERSION"))
            for row in result:
                print("Connected successfully. SQL Server version:")
                print(row[0])
            return engine
    except Exception as e:
        print("Connection failed:")
        print(e)
        return None

# Adjust to include error handling for the db connection method



In [205]:
def load_env():
    load_dotenv(dotenv_path="creds\\.env")


def SERVER_conn(input_site):

    load_env()

    # DB server
    site_server = os.getenv(input_site)
    
    
    paramz = {
        "site": os.getenv('site_server'),
        "userName": os.getenv('USER_NAME'),
        "Password": os.getenv('PASSWORD_dev-test'),
        "Driver": os.getenv("ODBC_DRIVER")
    }

    db = os.getenv(input_site)

    server_conn = establish_db_connection(
        paramz["site"],
        db, 
        paramz["userName"],     
        paramz["Password"],
        paramz["Driver"])
        
    return server_conn



def db_request(query, server_conn_str):
    if server_conn_str is None:
        raise Exception("Database connection failed. Please check your credentials and connection settings.")

    start_time = time.time()

    respo = pd.read_sql(query, server_conn_str)

    end_time = time.time()
    print(f"Query executed in {end_time - start_time:.2f} seconds")
    return respo 


In [ ]:
RO_all = "SELECT *  from Ops_tblRepairOrder where fldLastUpdated > '2023-01-1' AND fldStatus = 3 AND fldDivision IN (1)"
query_all_requests = "SELECT *  from Ops_tblRequests where fldLastUpdated > '2023-01-1' AND fldAddWorkStatus IN (100, 300, 400)" 
query_all_LabourLine = "SELECT *  from Ops_tblLabourLine where fldLastUpdated > '2023-01-1'"
query_all_PartsLine = "SELECT *  from Ops_tblPartsLine where fldLastUpdated > '2023-01-1'"

# More queries 




In [58]:
# search for op_codes_based_on_key_words

# select * from 
# Ops_tblOpCode2
# where fldDescription like ('%Water Pump%')

In [193]:

def pull_data_by_server(server_conn_str):
    # pull data for 
    RO_tbl = db_request(RO_all, vw_18_db)
    request_tbl = db_request(query_all_requests, vw_18_db)
    labourline_tbl = db_request(query_all_LabourLine, vw_18_db)
    partslines_tbl = db_request(query_all_PartsLine, vw_18_db)

    return RO_tbl, request_tbl, labourline_tbl, partslines_tbl


In [248]:
server_conn = SERVER_conn("DB_server_174")


Connected successfully. SQL Server version:
Microsoft SQL Server 2022 (RTM-CU21-GDR) (KB5068406) - 16.0.4222.2 (X64) 
	Oct  3 2025 16:55:17 
	Copyright (C) 2022 Microsoft Corporation
	Developer Edition (64-bit) on Windows Server 2022 Standard 10.0 <X64> (Build 20348: )



In [ ]:

# # pull data for 
# RO_tbl_vw_174 = db_request(RO_all, vw_18_db)
# request_tbl_vw_174 = db_request(query_all_requests, vw_18_db)
# labourline_tbl_vw_174 = db_request(query_all_LabourLine, vw_18_db)
# partslines_tbl_vw_174 = db_request(query_all_PartsLine, vw_18_db)



Query executed in 3.17 seconds
Query executed in 25.35 seconds
Query executed in 10.27 seconds
Query executed in 28.09 seconds


In [61]:

# function to drop empty columns
def drop_empty_columns(df):
    df_cleaning = df.copy()
    # drop empty columns - must all empty
    df_cleaning = df_cleaning.dropna(axis=1, how='all')
    
    return df_cleaning

In [62]:
# function to filter columns
def filter_for_essential_columns(df, essential_cols):
    df_selected = df[essential_cols].copy()
    return df_selected

In [ ]:
# Defined essential columns for each table

essential_columns_request_tbl = ['fldId', 'fldWorkItemRef', 'fldSequence', 'fldDescription',
       'fldRequestCodeRef', 'fldRequestCode', 'fldRequestedTime', 'fldOrderNumber',
        'fldLastUpdated']


essential_cols_labourline_tbl = ['fldID', 'fldRequestRef', 'fldOpCodeRef',
       'fldActualHours', 'fldSoldHours', 'fldDescription',
       'fldAddedDate', 'fldLastUpdated']

essential_cols_partlines_tbl = ['fldID', 'fldRequestRef', 'fldSequence', 'fldPartNumber', 'fldPartDesc',
       'fldRequested', 'fldShipped', 'fldOrderType', 'fldDateAdded',
       'fldLastUpdated']


essential_cols_RO_tbl = ['fldId', 'fldContactRef', 'fldVehicleRef', 'fldDateOpened',
       'fldDateClosed'
       ]
       
  

In [64]:

def clean_datset(df, tbl_type):
    df_dropped_empty_cols = drop_empty_columns(df)

    if tbl_type == "request":
        df_filtered = filter_for_essential_columns(df_dropped_empty_cols, essential_columns_request_tbl)
    
    elif tbl_type == "labourline":
        df_filtered = filter_for_essential_columns(df_dropped_empty_cols, essential_cols_labourline_tbl)

    elif tbl_type == "partslines":
        df_filtered = filter_for_essential_columns(df_dropped_empty_cols, essential_cols_partlines_tbl)

    elif tbl_type == "RO_tbl":
        df_filtered = filter_for_essential_columns(df_dropped_empty_cols, essential_cols_RO_tbl)
        # remove
        
    return df_filtered
 

In [65]:

request_tbl_vw_174 = clean_datset(request_tbl_vw_174, tbl_type="request")
labor_tbl_vw_174 = clean_datset(labourline_tbl_vw_174, tbl_type="labourline")
parts_tbl_vw_174 = clean_datset(partslines_tbl_vw_174, tbl_type="partslines")
RO_tbl_vw_174 = clean_datset(RO_tbl_vw_174, tbl_type="RO_tbl")

In [66]:

parts_tbl_vw_174.columns 
 

Index(['fldID', 'fldRequestRef', 'fldSequence', 'fldPartNumber', 'fldPartDesc',
       'fldRequested', 'fldShipped', 'fldOrderType', 'fldDateAdded',
       'fldLastUpdated'],
      dtype='object')

#### Find a list of labour and parts for the following repair jobs 

- water pump 
- Timing belt
- Electrical - exterior lights


In [262]:
def search_columns_for_keyword(df, keyword, column):
    if (column not in df.columns) or column=="":
        raise ValueError(f"Column '{column}' does not exist in the DataFrame.")
    filtered_df = df[df[column].str.contains(keyword, case=False, na=False)]
    return filtered_df

def get_top_ten_opcodes(df):
    top_ten = df["fldRequestCode"].value_counts().head(20)
    return top_ten

def search_request_by_opcode(df, opcode):
    search_result = df[df["fldRequestCode"]== opcode]
    
    return search_result

def search_request_by_list_of_opcodes(df, opcode_list):
    search_result = df[df["fldRequestCode"].isin(opcode_list)]
    
    return search_result

def search_parts_and_labour_by_req_id(labor, parts, req_id):
    labor_result = labor[labor["fldRequestRef"]== req_id]
    parts_result = parts[parts["fldRequestRef"]== req_id]
        
    print(f"Labour items for Request ID {req_id}:")
    display(labor_result)
    print(f"Parts items for Request ID {req_id}:")
    display(parts_result)
    

In [140]:

def search_op_code_by_keyword(db_conn, key_word):
    query = "select * from  Ops_tblOpCode2 where fldDescription like ('%"+key_word+"%')"
    search_result = db_request(query, db_conn)

    return search_result["fldCode"].values.tolist()

In [210]:

def clean_data(RO_tbl, request_tbl, labourline_tbl, partslines_tbl):
    RO_tbl_cleaned = clean_datset(RO_tbl, tbl_type="RO_tbl")
    request_tbl_cleaned = clean_datset(request_tbl, tbl_type="request")
    labourline_tbl_cleaned = clean_datset(labourline_tbl, tbl_type="labourline")
    partslines_tbl_cleaned = clean_datset(partslines_tbl, tbl_type="partslines")

    return RO_tbl_cleaned, request_tbl_cleaned, labourline_tbl_cleaned, partslines_tbl_cleaned


In [244]:
RO_tbl, request_tbl, labourline_tbl, partslines_tbl = pull_data_by_server(server_conn)


Query executed in 5.06 seconds
Query executed in 57.38 seconds
Query executed in 24.59 seconds
Query executed in 42.64 seconds


In [249]:
db_conn = server_conn
key_wrd = "Water Pump Replace"

In [253]:
RO_tbl, request_tbl, labourline_tbl, partslines_tbl = clean_data(RO_tbl, request_tbl, labourline_tbl, partslines_tbl)

(706234, 9)

In [261]:
opcodes_list = search_op_code_by_keyword(db_conn, key_wrd)
filtered_requests_df = search_request_by_list_of_opcodes(request_tbl, opcodes_list)
filtered_requests_df
# # search from labour line and part line where fldRequestRef in filtered_requests_df['fldId']
# filtered_labour_df = labourline_tbl[labourline_tbl["fldRequestRef"].isin(filtered_requests_df['fldId'])]
# filtered_parts_df = partslines_tbl[partslines_tbl["fldRequestRef"].isin(filtered_requests_df['fldId'])]


# parts_summary_df=[]
# labour_summary_df=[]
# request_summary_df=[]

# parts_summary_df = pd.DataFrame(columns=["Part Number", "Part Description", "Occurrence_count"])
# labour_summary_df = pd.DataFrame(columns=["fldOpCodeRef", "fldDescription", "Occurrence_count"])
# request_summary_df = pd.DataFrame(columns=["Request ID", "Description","fldRequestCode", "#_PartItems", "#_LaborItems"])

Query executed in 0.02 seconds
['09Z35']


,fldId,fldWorkItemRef,fldSequence,fldDescription,fldRequestCodeRef,fldRequestCode,fldRequestedTime,fldOrderNumber,fldLastUpdated


In [257]:
filtered_parts_df

,fldID,fldRequestRef,fldSequence,fldPartNumber,fldPartDesc,fldRequested,fldShipped,fldOrderType,fldDateAdded,fldLastUpdated


In [ ]:

# def maximus(db_conn, key_wrd, RO_tbl, request_tbl, labourline_tbl, partslines_tbl):


    for items in filtered_requests_df['fldId'].values:
        req_id = items

        tracker_count_part_item_once_per_job = set()
        tracker_count_labour_item_once_par_job = set()

    # print(filtered_requests_df[filtered_requests_df["fldId"] == req_id ]["fldDescription"])

        # search for all parts and labour lines for this req_id
        part_items = filtered_parts_df[filtered_parts_df["fldRequestRef"]== req_id]
        labour_items = filtered_labour_df[filtered_labour_df["fldRequestRef"]==req_id]

        new_row = {
            "Request ID": req_id,
            "Description": filtered_requests_df[filtered_requests_df["fldId"] == req_id ]["fldDescription"].values[0],
            "fldRequestCode": filtered_requests_df[filtered_requests_df["fldId"] == req_id ]["fldRequestCode"].values[0],
            "#_PartItems": len(part_items),
            "#_LaborItems": len(labour_items)
            }

        request_summary_df = pd.concat([request_summary_df, pd.DataFrame([new_row])], ignore_index=True )

        if not part_items.empty:
            for index, row in part_items.iterrows():
                part_number = row["fldPartNumber"]
                part_desc = row["fldPartDesc"]

                
                if part_number in parts_summary_df["Part Number"].values and part_number not in tracker_count_part_item_once_per_job:
                    parts_summary_df.loc[parts_summary_df["Part Number"] == part_number, "Occurrence_count"] += 1
                    tracker_count_part_item_once_per_job.add(part_number)
                else:
                    new_row = {
                        "Part Number": part_number,
                        "Part Description": part_desc,
                        "Occurrence_count": 1
                    }
                    parts_summary_df = pd.concat([parts_summary_df, pd.DataFrame([new_row])], ignore_index=True) 
                    tracker_count_part_item_once_per_job.add(part_number)
                    parts_summary_df.sort_values(by="Occurrence_count", ascending=False, inplace=True)

                
        if not labour_items.empty:
            for index, row in labour_items.iterrows():
                op_code = row["fldOpCodeRef"]
                labour_desc = row["fldDescription"]

                if op_code in labour_summary_df["fldOpCodeRef"].values and op_code not in tracker_count_labour_item_once_par_job:
                    labour_summary_df.loc[labour_summary_df["fldOpCodeRef"] == op_code, "Occurrence_count"] += 1
                    tracker_count_labour_item_once_par_job.add(op_code)
                else:
                    new_row = {
                        "fldOpCodeRef": op_code,
                        "fldDescription": labour_desc,
                        "Occurrence_count": 1
                    }
                    labour_summary_df = pd.concat([labour_summary_df, pd.DataFrame([new_row])] , ignore_index=True )
                    tracker_count_labour_item_once_par_job.add(op_code)
                    labour_summary_df.sort_values(by="Occurrence_count", ascending=False, inplace=True)

    return request_summary_df, parts_summary_df, labour_summary_df

In [ ]:
RO_tbl.shape

(64350, 78)

In [250]:

request_summary_df, parts_summary_df, labour_summary_df = maximus(server_conn, 
                                                                    "Water Pump Replace", 
                                                                    RO_tbl, 
                                                                    request_tbl, 
                                                                    labourline_tbl, 
                                                                    partslines_tbl)

                                                                    # RO_tbl, request_tbl, labourline_tbl, partslines_tbl
 

Query executed in 0.02 seconds


In [251]:
# filter for the keyword

print("Requests Summary")

display(request_summary_df[request_summary_df["Description"]=="WATER PUMP REPLACE"])


print("Parts Summary")
display(parts_summary_df)


print("Labour Summary")
display(labour_summary_df)

Requests Summary


,Request ID,Description,fldRequestCode,#_PartItems,#_LaborItems


Parts Summary


,Part Number,Part Description,Occurrence_count


Labour Summary


,fldOpCodeRef,fldDescription,Occurrence_count


In [185]:
# Labour and Parts by Request ID 

# req_id="361C71BE-99B9-4CF8-8FA4-2211554A56EA"
req_id="034B0D9F-F542-4B68-9335-31C05279811B"

search_parts_and_labour_by_req_id(waterPump_labour_df, waterPump_parts_df, req_id= req_id)
 

Labour items for Request ID 034B0D9F-F542-4B68-9335-31C05279811B:


,fldID,fldRequestRef,fldOpCodeRef,fldActualHours,fldSoldHours,fldDescription,fldAddedDate,fldLastUpdated
54616,241A9CED-9C08-4510-958A-C48E31DAA65D,034B0D9F-F542-4B68-9335-31C05279811B,D8A5EB04-DD37-45E1-BC68-0BA0CB543ACB,3.18,4.0,WATER PUMP REPLACE,2025-06-18,2025-06-19 12:52:40.817


Parts items for Request ID 034B0D9F-F542-4B68-9335-31C05279811B:


,fldID,fldRequestRef,fldSequence,fldPartNumber,fldPartDesc,fldRequested,fldShipped,fldOrderType,fldDateAdded,fldLastUpdated
80149,CB86A33B-1FE1-4317-B74A-F6848A22F98E,034B0D9F-F542-4B68-9335-31C05279811B,1,022 145 933 AP,RIBBEDBELT,1.0,1.0,0,2025-06-18 20:56:45.683,2025-06-19 12:52:40.817
80150,463D9405-E098-4D3C-91CE-5FBECA173EA3,034B0D9F-F542-4B68-9335-31C05279811B,2,03H 121 008 M,WATER PUMP,1.0,1.0,0,2025-06-18 20:56:58.247,2025-06-19 12:52:40.817
80151,CDC31BB0-BE4C-466E-BDE2-75A37E6957AC,034B0D9F-F542-4B68-9335-31C05279811B,3,N 105 268 02,SCREW,3.0,3.0,0,2025-06-18 20:57:03.127,2025-06-19 12:52:40.817
80152,0D33A965-A06F-43F9-8302-79B9D3358D4A,034B0D9F-F542-4B68-9335-31C05279811B,4,N 910 214 03,BOLT,3.0,3.0,0,2025-06-18 20:57:14.757,2025-06-19 12:52:40.817
80153,74B7DDAF-1592-46A7-BAFA-643BD14905AF,034B0D9F-F542-4B68-9335-31C05279811B,5,N 107 847 01,BOLT,1.0,1.0,0,2025-06-18 20:57:27.040,2025-06-19 12:52:40.817
80154,B2328354-FE3C-4B66-9D55-E4B36286933A,034B0D9F-F542-4B68-9335-31C05279811B,6,N 107 848 01,SCREW,1.0,1.0,0,2025-06-18 20:57:40.120,2025-06-19 12:52:40.817
80155,A18AFBEF-CF2E-4444-8D43-DE3E8A406BA9,034B0D9F-F542-4B68-9335-31C05279811B,7,N 105 524 04,BOLT,2.0,2.0,0,2025-06-19 16:06:08.190,2025-06-19 12:52:40.817
80156,9B25A995-A849-4EF6-B362-D3AE29857925,034B0D9F-F542-4B68-9335-31C05279811B,8,N 107 069 01,BOLT,1.0,1.0,0,2025-06-19 16:06:10.127,2025-06-19 12:52:40.817
80157,DB9F3632-20EC-4E48-B7D2-947F2E8D4F59,034B0D9F-F542-4B68-9335-31C05279811B,9,N 910 296 02,BOLT,1.0,1.0,0,2025-06-19 16:06:10.910,2025-06-19 12:52:40.817
80158,B2C179D1-6236-42EC-A1BB-7B0FDE44695B,034B0D9F-F542-4B68-9335-31C05279811B,10,N 912 040 01,SCREW,1.0,1.0,0,2025-06-19 16:06:11.627,2025-06-19 12:52:40.817


In [71]:


# for row in water_pump_req_df.itertuples():
#     req_id = row.fldId
#     labor_lines, part_lines = search_parts_and_labour_by_req_id(waterPump_labour_df, waterPump_parts_df, req_id)
    
#     print(f"Request ID: {req_id}")
#     print("Labor Lines:")
#     print(labor_lines)
#     print("\nParts Lines:")
#     print(part_lines)
#     print("\n" + "="*50 + "\n")


In [72]:
# create a summary df that summarizes the data
# list all the unique requests from req table and save them in a df
# Go through the parts and labour tables and count the number of part and labour items associated with that request id
# this will provide insight on the difference between the number of items suggested for the job


# create a summary of occurrence of part and labour items
# by partNumber, count how many time each part number appears and rank them by how often they appear 



In [ ]:
# Pull data

site_number ='130'



